In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("../UPDATED_NLP_COURSE/TextFiles/smsspamcollection.tsv",sep='\t')

In [3]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [5]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [6]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = df["message"]
y = df['label']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
count_vect = CountVectorizer()

In [12]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
5       FreeMsg Hey there darling it's been 3 week's n...
6       Even my brother is not like to speak with me. ...
7       As per your request 'Melle Melle (Oru Minnamin...
8       WINNER!! As a valued network customer you have...
9       Had your mobile 11 months or more? U R entitle...
10      I'm gonna be home soon and i don't want to tal...
11      SIX chances to win CASH! From 100 to 20,000 po...
12      URGENT! You have won a 1 week FREE membership ...
13      I've been searching for the right words to tha...
14                    I HAVE A DATE ON SUNDAY WITH WILL!!
15      XXXMobileMovieClub: To use your credit, click ...
16                             Oh k...i'm watching here:)
17      Eh u r

In [13]:
X_train_counts = count_vect.fit_transform(X_train)

In [14]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [16]:
X_train.shape

(3733,)

In [17]:
X_train_counts.shape

(3733, 7082)

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer

In [19]:
tfidf_transformer = TfidfTransformer()

In [20]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [21]:
X_train_tfidf.shape

(3733, 7082)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
vectorizer = TfidfVectorizer()

In [24]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [25]:
from sklearn.svm import LinearSVC

In [26]:
clf = LinearSVC()

In [27]:
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [28]:
from sklearn.pipeline import Pipeline

In [29]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [30]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [31]:
predictions = text_clf.predict(X_test)

In [32]:
X_test

3245    Squeeeeeze!! This is christmas hug.. If u lik ...
944     And also I've sorta blown him off a couple tim...
1044    Mmm thats better now i got a roast down me! i...
2484        Mm have some kanji dont eat anything heavy ok
812     So there's a ring that comes with the guys cos...
2973    Sary just need Tim in the bollox &it hurt him ...
2991    Love isn't a decision, it's a feeling. If we c...
2942    My supervisor find 4 me one lor i thk his stud...
230                    Dear good morning now only i am up
1181                           I'm in chennai velachery:)
1912    Lol grr my mom is taking forever with my presc...
1992    No other Valentines huh? The proof is on your ...
5435                    I'm wif him now buying tix lar...
4805    Er, hello, things didn‘t quite go to plan – is...
401     FREE RINGTONE text FIRST to 87131 for a poly o...
1859                     Sir, i am waiting for your call.
1344    Crazy ar he's married. Ü like gd looking guys ...
2952          

In [33]:
from sklearn.metrics import confusion_matrix,classification_report

In [34]:
print(confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [35]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [39]:
from sklearn import metrics

In [40]:
metrics.accuracy_score(y_test,predictions)

0.989668297988037

In [41]:
text_clf.predict(["Hi how are you doing today?"])

array(['ham'], dtype=object)

In [48]:
text_clf.predict(["Congratulations! You've been selected as a winner. Text WON to 24324"])

array(['spam'], dtype=object)